In [26]:
import torch
import numpy as np
from torch import nn
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
import torch.optim as optim
from torch.nn import Sequential, ReLU, Linear
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

device = "cuda" if torch.cuda.is_available() else "cpu"


### Preparação do dataset

In [5]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

data.train_mask = data.val_mask = data.test_mask = None

#Note o detalhe para add_negative_train_samples=True (adiciona exemplos de nós que não possuem arestas)
transform = T.RandomLinkSplit(is_undirected=True, add_negative_train_samples=True)
train_data, val_data, test_data = transform(data)
train_data = train_data.to(device)
val_data=val_data.to(device)
test_data = test_data.to(device)
print(test_data)

print(train_data.edge_label)

Data(x=[2708, 1433], edge_index=[2, 8446], y=[2708], edge_label=[2110], edge_label_index=[2, 2110])
tensor([1., 1., 1.,  ..., 0., 0., 0.])


In [6]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708])

#### Nota:

Aqui estamos usando um dataset pronto que usou a interface 'dataset'. Veja comentários no notebook de Pytorch Geometric sobre possibilidades alternativas para montar o dataset.

### Definição do modelo

In [9]:
class Feedforward(torch.nn.Module):
    
        def __init__(self, input_size, hidden_size):
            super(Feedforward, self).__init__()
            
            self.input_size = input_size
            self.hidden_size  = hidden_size
            
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.fc2 = torch.nn.Linear(self.hidden_size, self.hidden_size)
            self.fc3 = torch.nn.Linear(self.hidden_size, self.hidden_size)
            self.fc4 = torch.nn.Linear(self.hidden_size, 1)
            
            self.relu = torch.nn.ReLU()
            self.out_act = nn.Sigmoid()
            
            
        def forward(self, x):
            output = self.fc1(x)
            output = self.relu(output)
            
            output = self.fc2(output)
            output = self.relu(output)
            
            output = self.fc3(output)
            output = self.relu(output)
            
            output = self.fc4(output)
            
            output = self.out_act(output)

            return output

### Criando features que representam as arestas: soma das features dos nós

In [7]:
num_features = train_data.x.shape[1]

## Gerando as features para o treino
featuresEdges_treino = ""

totalEdges_treino = train_data.edge_label_index.shape[1]

for col in range(0,totalEdges_treino):
    node1 = train_data.edge_label_index[0,col]
    node2 = train_data.edge_label_index[1,col]
    
    vals1 = train_data.x[[node1,]]
    vals1 = torch.reshape(vals1, (1, num_features))
    
    vals2 = train_data.x[[node2,]]
    vals2 = torch.reshape(vals2, (1, num_features))
    
    if col == 0:
        featuresEdges_treino = vals1+vals2
        continue
        
    somado = vals1+vals2
    
    featuresEdges_treino =  torch.cat((featuresEdges_treino, somado), dim=0) 


##########################################################################################
##Gerando as features para o teste
featuresEdges_teste = ""

totalEdges_teste = test_data.edge_label_index.shape[1]
print(totalEdges_teste)
for col in range(0,totalEdges_teste):

    node1 = test_data.edge_label_index[0,col]
    node2 = test_data.edge_label_index[1,col]
    
    vals1 = test_data.x[[node1,]]
    vals1 = torch.reshape(vals1, (1, num_features))

    vals2 = test_data.x[[node2,]]
    vals2 = torch.reshape(vals2, (1, num_features))
    
    if col == 0:
        featuresEdges_teste = vals1+vals2
        continue
        
    somado = vals1+vals2
    
    featuresEdges_teste =  torch.cat((featuresEdges_teste, somado), dim=0) 

print(featuresEdges_teste.shape)
print(featuresEdges_treino.shape)

2110
torch.Size([2110, 1433])
torch.Size([7392, 1433])


### Instância do modelo e outros parâmetros

In [10]:
num_features = train_data.x.shape[1]
hidden_channel = 64

model = Feedforward(num_features, hidden_channel).to(device)
print(model)
criterion = torch.nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

Feedforward(
  (fc1): Linear(in_features=1433, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (out_act): Sigmoid()
)


### Treino e teste

In [11]:
model.eval()
y_pred = model(featuresEdges_treino)
train_data.edge_label = torch.reshape(train_data.edge_label,(-1,1))

antes_treino = criterion(y_pred, train_data.edge_label) 

print('Teste - perda antes do treinamento' , antes_treino.item())

model.train()
epoch = 1000

for epoch in range(epoch):
    optimizer.zero_grad()
    
    # Passe Forward
    y_pred = model(featuresEdges_treino)
    
    # Computa a perda
    loss = criterion(y_pred, train_data.edge_label)
    
    print('Epoch {}: perda treino: {}'.format(epoch, loss.item()))
    
    # Passe de Backward
    loss.backward()
    optimizer.step()


Teste - perda antes do treinamento 0.6932457685470581
Epoch 0: perda treino: 0.6932457685470581
Epoch 1: perda treino: 0.6886391043663025
Epoch 2: perda treino: 0.6709862351417542
Epoch 3: perda treino: 0.6469810009002686
Epoch 4: perda treino: 0.6615766882896423
Epoch 5: perda treino: 0.6035478115081787
Epoch 6: perda treino: 0.6120639443397522
Epoch 7: perda treino: 0.5741829872131348
Epoch 8: perda treino: 0.5759062170982361
Epoch 9: perda treino: 0.5338155031204224
Epoch 10: perda treino: 0.5342048406600952
Epoch 11: perda treino: 0.4896065890789032
Epoch 12: perda treino: 0.4978921711444855
Epoch 13: perda treino: 0.4497186541557312
Epoch 14: perda treino: 0.4346380829811096
Epoch 15: perda treino: 0.4019308388233185
Epoch 16: perda treino: 0.36785611510276794
Epoch 17: perda treino: 0.35138803720474243
Epoch 18: perda treino: 0.29474809765815735
Epoch 19: perda treino: 0.2807169556617737
Epoch 20: perda treino: 0.2647995352745056
Epoch 21: perda treino: 0.2083253562450409
Epoch 2

Epoch 181: perda treino: 3.053592308788211e-06
Epoch 182: perda treino: 3.0311769023683155e-06
Epoch 183: perda treino: 3.0091325697867433e-06
Epoch 184: perda treino: 2.9873942821723176e-06
Epoch 185: perda treino: 2.965970907098381e-06
Epoch 186: perda treino: 2.944878360722214e-06
Epoch 187: perda treino: 2.9240518415463157e-06
Epoch 188: perda treino: 2.9036527848802507e-06
Epoch 189: perda treino: 2.883310344259371e-06
Epoch 190: perda treino: 2.863500185412704e-06
Epoch 191: perda treino: 2.8438030312827323e-06
Epoch 192: perda treino: 2.8242109237908153e-06
Epoch 193: perda treino: 2.8049814773112303e-06
Epoch 194: perda treino: 2.786147433653241e-06
Epoch 195: perda treino: 2.7671762836689595e-06
Epoch 196: perda treino: 2.7486084945849143e-06
Epoch 197: perda treino: 2.7304681680107024e-06
Epoch 198: perda treino: 2.712513833103003e-06
Epoch 199: perda treino: 2.6948493996314937e-06
Epoch 200: perda treino: 2.6770799195219297e-06
Epoch 201: perda treino: 2.659924120962387e-06


Epoch 353: perda treino: 1.2729451555060223e-06
Epoch 354: perda treino: 1.2681143743975554e-06
Epoch 355: perda treino: 1.263097942683089e-06
Epoch 356: perda treino: 1.2582993349496974e-06
Epoch 357: perda treino: 1.2535248288259027e-06
Epoch 358: perda treino: 1.248798866981815e-06
Epoch 359: perda treino: 1.244064492311736e-06
Epoch 360: perda treino: 1.239330572389008e-06
Epoch 361: perda treino: 1.2347255733402562e-06
Epoch 362: perda treino: 1.2299833542783745e-06
Epoch 363: perda treino: 1.225297751261678e-06
Epoch 364: perda treino: 1.2208056432427838e-06
Epoch 365: perda treino: 1.2161278846178902e-06
Epoch 366: perda treino: 1.2116922789573437e-06
Epoch 367: perda treino: 1.2071677701896988e-06
Epoch 368: perda treino: 1.2026434887957294e-06
Epoch 369: perda treino: 1.1981270517935627e-06
Epoch 370: perda treino: 1.1936027703995933e-06
Epoch 371: perda treino: 1.189167051052209e-06
Epoch 372: perda treino: 1.1848040912809665e-06
Epoch 373: perda treino: 1.1803927009168547e-0

Epoch 528: perda treino: 7.170444860093994e-07
Epoch 529: perda treino: 7.151977570174495e-07
Epoch 530: perda treino: 7.132382506824797e-07
Epoch 531: perda treino: 7.112785738172533e-07
Epoch 532: perda treino: 7.093754561537935e-07
Epoch 533: perda treino: 7.073835490700731e-07
Epoch 534: perda treino: 7.054320576571627e-07
Epoch 535: perda treino: 7.034482791823393e-07
Epoch 536: perda treino: 7.015370329099824e-07
Epoch 537: perda treino: 6.997145192144671e-07
Epoch 538: perda treino: 6.978114015510073e-07
Epoch 539: perda treino: 6.959405709494604e-07
Epoch 540: perda treino: 6.941341439414828e-07
Epoch 541: perda treino: 6.922230113559635e-07
Epoch 542: perda treino: 6.903359803800413e-07
Epoch 543: perda treino: 6.884086474201467e-07
Epoch 544: perda treino: 6.865539035061374e-07
Epoch 545: perda treino: 6.846749442956934e-07
Epoch 546: perda treino: 6.82973393395514e-07
Epoch 547: perda treino: 6.809492560932995e-07
Epoch 548: perda treino: 6.792396902710607e-07
Epoch 549: per

Epoch 707: perda treino: 4.638749260266195e-07
Epoch 708: perda treino: 4.6283471988317615e-07
Epoch 709: perda treino: 4.619315632226062e-07
Epoch 710: perda treino: 4.609800328125857e-07
Epoch 711: perda treino: 4.5998817199688347e-07
Epoch 712: perda treino: 4.589882678374124e-07
Epoch 713: perda treino: 4.5803673742739193e-07
Epoch 714: perda treino: 4.570287330807332e-07
Epoch 715: perda treino: 4.5602880049955274e-07
Epoch 716: perda treino: 4.549886227778188e-07
Epoch 717: perda treino: 4.540290206023201e-07
Epoch 718: perda treino: 4.5313390728551894e-07
Epoch 719: perda treino: 4.521097878296132e-07
Epoch 720: perda treino: 4.51182444294318e-07
Epoch 721: perda treino: 4.502228705405287e-07
Epoch 722: perda treino: 4.4923908149030467e-07
Epoch 723: perda treino: 4.4834399659521296e-07
Epoch 724: perda treino: 4.4734412085745134e-07
Epoch 725: perda treino: 4.4644093577517197e-07
Epoch 726: perda treino: 4.4540075805343804e-07
Epoch 727: perda treino: 4.444895012056804e-07
Epoc

Epoch 880: perda treino: 3.30347063481895e-07
Epoch 881: perda treino: 3.297181194739096e-07
Epoch 882: perda treino: 3.291455357157247e-07
Epoch 883: perda treino: 3.2857303722266806e-07
Epoch 884: perda treino: 3.280811711192655e-07
Epoch 885: perda treino: 3.2748445732977416e-07
Epoch 886: perda treino: 3.2699256280466216e-07
Epoch 887: perda treino: 3.2629100132908206e-07
Epoch 888: perda treino: 3.256701290865749e-07
Epoch 889: perda treino: 3.251217890465341e-07
Epoch 890: perda treino: 3.246218511776533e-07
Epoch 891: perda treino: 3.240089938572055e-07
Epoch 892: perda treino: 3.234767973481212e-07
Epoch 893: perda treino: 3.228317098091793e-07
Epoch 894: perda treino: 3.2234788704954553e-07
Epoch 895: perda treino: 3.216544257611531e-07
Epoch 896: perda treino: 3.2112217240864993e-07
Epoch 897: perda treino: 3.2042873954196693e-07
Epoch 898: perda treino: 3.198400690962444e-07
Epoch 899: perda treino: 3.1929175747791305e-07
Epoch 900: perda treino: 3.1875151762505993e-07
Epoch

In [36]:
model.eval()

y_pred = model(featuresEdges_teste)
y_pred = torch.reshape(y_pred,(-1,1))

threshold = torch.tensor([0.5]).to("cpu")

#Atribui 0 ou 1 de acordo com o threshold
results = (y_pred>threshold).float()*1

#reshape
test_data.edge_label = torch.reshape(test_data.edge_label,(-1,1))

#Area under the curve - AUC
print(roc_auc_score(test_data.edge_label.cpu(),  y_pred.detach().numpy()))

#Computes - F1 score
print(f1_score(test_data.edge_label.cpu(),results.cpu()))


0.7345230340738079
0.7071362372567193
